## notes for what the function should take
bootstrap_number, default is 100


network metric, either between centrality, connectivity


top and bottom which percent (default 10%)

current working directory


In [1]:
import pandas as pd
from itertools import product
import numpy as np
from collections import Counter
import collections
from numpy import random as rd
from SFS import SFS
import os
import re


In [62]:
#deleting all to start a new bootstrap 
# !rm -r conn
!rm -r run*
!rm -r upper*
!rm -r lower*
!rm -r 0*
!rm -r bet*
!rm -r dfe_outfile_*
!rm -r output.txt
!rm -r prop_muts_in_s_ranges.out
!rm -r low_conn*
!rm -r alpha_omega_a_between_centrality.png
!rm -r random*

rm: cannot remove `0*': No such file or directory
rm: cannot remove `bet*': No such file or directory
rm: cannot remove `dfe_outfile_*': No such file or directory
rm: cannot remove `output.txt': No such file or directory
rm: cannot remove `prop_muts_in_s_ranges.out': No such file or directory
rm: cannot remove `low_conn*': No such file or directory
rm: cannot remove `alpha_omega_a_between_centrality.png': No such file or directory
rm: cannot remove `random*': No such file or directory


In [2]:
os.mkdir("./runs")
for i in range(1,101):
    os.mkdir("./runs/run" + str(i))

In [3]:
def get_dfe_outputs(filename, t, network_measure):
    with open(filename) as f:
        content = f.readlines()[0]
        print(content)
        match = re.search('lambda (\d+.\d+) selected_divergence (\d+.\d+) alpha (\d+.\d+) omega_A (\d+.\d+)', content)
        print("this is match " + str(match))
        with open(t+"_" +network_measure+'_lambda.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(1)))
            print("updated file: " + str(the_file))
        with open(t+"_" +network_measure+'_selected_divergence.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(2)))
            print("updated file: " + str(the_file))
        with open(t+"_" +network_measure+'_alpha.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(3)))
            print("updated file: " + str(the_file))
        with open(t+"_" +network_measure+'_omega_A.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(4)))
            print("updated file: " + str(the_file))
        with open(t+"_" +network_measure+'_all.csv', 'a') as the_file:
            the_file.write("{}\t{}\t{}\t{}\t \n".format(match.group(1), match.group(2), match.group(3), match.group(4)))
            print("updated file: " + str(the_file))
#             df = pd.read_csv(t+"_" +network_measure+'_all.csv', header=None, delimiter="\t")
#             print("here is the df ")
#             print(df.head)
#             df.rename(columns={0: 'lambda_'+ t, 1: 'selected_divergence_'+ t, 2:'alpha_' +t, 3:'omega_A_'+t}, inplace=True)
#             df.to_csv(t+"_" +network_measure+'_all.csv', index=False)
            
            
            
            
#             ##to add column names
#             df = pd.read_csv(t+"_" +network_measure+'_all.csv', header=None, delimiter="\t")
#             print("here is the df ")
#             print(df.head)
#             df.columns = ['lambda_'+ t, 'selected_divergence_'+ t, 'alpha_' +t, 'omega_A_'+t]
#             df.to_csv(t+"_" +network_measure+'_all.csv')
        

In [4]:
import pandas as pd
df = pd.read_csv('connectivity_bc_irc1080_final_04-09-2020.csv', sep=",")
print(df.columns)

Index(['Unnamed: 0', 'ness_ID', 'Cincerta_transcript_ID', 'PID',
       'aligned_sites', 'k0', 'k4', 'diffs0', 'sites0', 'diffs4', 'sites4',
       'gene', 'transcript', 'SFS0', 'SFS4', 'pi0', 'pi4', 'max_connectivity',
       'BetweennessCentrality'],
      dtype='object')


In [5]:
%matplotlib inline


def sum_SFSs(list_of_SFSs):
    # for a bunch of SFSs that are the same length sum them alllll up into one big one\
#     sfs_size = len(list_of_SFSs[0])
#     for i in range(sfs_size):
#         sum([sfs[i] for sfs in list_of_SFSs ]
    total_SFS = [sum(i) for i in zip(*list_of_SFSs)]
    return total_SFS

def column2SFSs(column_of_SFSs):
    list_of_SFSs = []
    for s in list(column_of_SFSs):
        exec("list_of_SFSs.append(" + s + ")")
    return list_of_SFSs

def make_dfe_inputs(lower_path, upper_path):
    df = pd.read_csv('connectivity_bc_irc1080_final_04-09-2020.csv', sep=",")
    
    x = "BetweennessCentrality"
    gene_name = "gene"
    cutoff_percent  = 0.10
    percentage = "10"
    number_genes = df.shape[0]
    cutoff_number = round(number_genes*cutoff_percent)
    cutoff_high  = sorted(list(df[x]))[number_genes-cutoff_number]
    cutoff_low = sorted(list(df[x]))[cutoff_number]

    df_low = df[df[x] <= cutoff_low]
    df_high = df[df[x] >= cutoff_high]
    

    df_low_names = df_low[gene_name].tolist()
    random_low_lst = (list(rd.choice(df_low_names, size=100, replace=True)))  # cleaned = df[df['genename'].isin(keep_lst)] #ignores duplicates so this is wrong 
    low_newDF = pd.DataFrame() #creates a new dataframe that's empty
    for key,value in (dict(Counter(random_low_lst)).items()):
        for i in range(value):
                low_newDF = low_newDF.append(df_low.loc[df_low[gene_name] == key]) # ignoring index is optional

                
    df_high_names = df_high[gene_name].tolist()
    random_high_lst = (list(rd.choice(df_high_names, size=100, replace=True)))
    high_newDF = pd.DataFrame() #creates a new dataframe that's empty
    for key,value in (dict(Counter(random_high_lst)).items()):
        for i in range(value):
                high_newDF = high_newDF.append(df_high.loc[df_high[gene_name] == key]) # ignoring index is optional
        

    total_SFS0_low = SFS(sum_SFSs(column2SFSs(low_newDF.SFS0)))
    total_SFS4_low = SFS(sum_SFSs(column2SFSs(low_newDF.SFS4)))
    total_SFS0_high = SFS(sum_SFSs(column2SFSs(high_newDF.SFS0)))
    total_SFS4_high= SFS(sum_SFSs(column2SFSs(high_newDF.SFS4)))

    SFSs = [total_SFS0_low,total_SFS4_low,total_SFS0_high,total_SFS4_high]
    
    for s in SFSs:
        s.fold()

    sfs_lower_network_measure = "SFS.lower" + str(percentage) + "th_" + str(network_measure) + ".txt"
    SFS_lower_0 = " ".join(str(item) for item in SFSs[0].sfs)
    SFS_lower_1 = " ".join(str(item) for item in SFSs[1].sfs)
    !printf "1\n18\n" > $lower_path/$sfs_lower_network_measure
    !echo $SFS_lower_0 >> $lower_path/$sfs_lower_network_measure
    !echo $SFS_lower_1 >> $lower_path/$sfs_lower_network_measure
    
    
    sfs_upper_network_measure = "SFS.upper" + str(percentage) + "th_" + str(network_measure) + ".txt"
    SFS_upper_0 = " ".join(str(item) for item in SFSs[2].sfs)
    SFS_upper_1 = " ".join(str(item) for item in SFSs[3].sfs)
    !printf "1\n18\n" > $upper_path/$sfs_upper_network_measure
    !echo $SFS_upper_0 >> $upper_path/$sfs_upper_network_measure
    !echo $SFS_upper_1 >> $upper_path/$sfs_upper_network_measure
    
    divergence_lower_network_measure = "divergence.lower" +str(percentage) + "th_" + str(network_measure) + ".txt"
    low_sites_0 = sum(low_newDF.sites0)
    low_diff_0 = sum(low_newDF.diffs0)
    low_sites_4 = sum(low_newDF.sites4)
    low_diff_4 = sum(low_newDF.diffs4)
    !echo 1 $low_sites_0 $low_diff_0 > $lower_path/$divergence_lower_network_measure
    !echo 0 $low_sites_4 $low_diff_4 >> $lower_path/$divergence_lower_network_measure

    divergence_upper_network_measure = "divergence.upper" + str(percentage) + "th_" + str(network_measure) + ".txt"
    high_sites_0 = sum(high_newDF.sites0)
    high_diff_0 = sum(high_newDF.diffs0)
    high_sites_4 = sum(high_newDF.sites4)
    high_diff_4 = sum(high_newDF.diffs4)
    !echo 1 $high_sites_0 $high_diff_0 > $upper_path/$divergence_upper_network_measure
    !echo 0 $high_sites_4 $high_diff_4 >> $upper_path/$divergence_upper_network_measure
    
    


In [7]:
print(os.getcwd())

C:\Users\selsh\repos\Network-Evolution


In [10]:
%%time
import os
network_measure = "irc1080_betweenness_centrality"
# rootdir = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs" 
# rootdir_runs = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs" 
rootdir = r"C:\Users\selsh\repos\Network-Evolution\runs"
rootdir_runs = r"C:\Users\selsh\repos\Network-Evolution\runs"
# rootdir_org = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/"
rootdir_org = r"C:\Users\selsh\repos\Network-Evolution"
print(rootdir)
i = 0
upper = "upper10th"
lower = "lower10th"
# network_measure = "chlamynet_betweenness_centr"
for subdir, dirs, files in os.walk(rootdir):
    for folder in dirs:
        if folder.startswith("r"):
            new_path = os.path.join(subdir, folder)
            print("this is the path " + str(new_path))
            os.chdir(new_path) #change directory to new_path 
            if not os.path.exists(upper) and not os.path.exists(lower):
                os.makedirs(upper)
                os.makedirs(lower)
                
            #make folders in lower folder for each bootstrap   
            os.chdir(new_path + "/" + lower) #change directory to new_path
            if not os.path.exists(lower + "_" + network_measure + "_neutral") and not os.path.exists(lower + "_" + network_measure + "_selected"):
                os.makedirs(lower + "_" + network_measure + "_neutral")
                os.makedirs(lower + "_" + network_measure + "_selected")
            
            os.chdir(new_path + "/" + upper) #change directory to new_path
            
            if not os.path.exists(upper + "_" + network_measure + "_neutral") and not os.path.exists(upper + "_" + network_measure + "_selected"):
                os.makedirs(upper + "_" + network_measure + "_neutral")
                os.makedirs(upper + "_" + network_measure + "_selected")



#             send the two paths (upper10th and lower10th to the function, to enter divergence and diversity )
            os.chdir(rootdir_org)
            make_dfe_inputs(new_path + "/" + lower, new_path + "/" + upper)
            
            data_path_1 = "/scratch/research/tmp_apps/dfe-alpha-release-2.16/data/"
            runs_folder = rootdir_runs
#             runs_folder = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs"
            print(os.getcwd())
            
            
            ############ class 0 lower and upper, NEUTRAL          
            class0_config_lower_filename = '{}/{}/{}/run.{}.est_dfe-site_class0.config.txt'.format(runs_folder, folder, lower, lower)
            f = open(class0_config_lower_filename, 'w')
            f.write("data_path_1\t{} \nsfs_input_file \t    {}/{}/{}/SFS.{}_{}.txt \nest_dfe_results_dir \t   {}/{}/{}/{}_{}_neutral \nsite_class 0 \nfold       1 \nepochs     2 \nsearch_n2       1 \nt2_variable     1 \nt2              50".format(data_path_1, runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, lower, network_measure))
            f.close()
            
            #outputs info in neutral folder in lower (neut_exp_obs_allele_freqs.csv)
            !bash dfe_first_step.sh $class0_config_lower_filename
            
            #do the same for upper: 
            class0_config_upper_filename = '{}/{}/{}/run.{}.est_dfe-site_class0.config.txt'.format(runs_folder, folder, upper, upper)
            f = open(class0_config_upper_filename, 'w')
            f.write("data_path_1\t{} \nsfs_input_file \t    {}/{}/{}/SFS.{}_{}.txt \nest_dfe_results_dir \t   {}/{}/{}/{}_{}_neutral \nsite_class 0 \nfold       1 \nepochs     2 \nsearch_n2       1 \nt2_variable     1 \nt2              50".format(data_path_1, runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, upper, network_measure))
            f.close()
            
            #outputs info in neutral folder in upper (neut_exp_obs_allele_freqs.csv)
            !bash dfe_first_step.sh $class0_config_upper_filename
            
            
            ###########class 1 lower and upper, SELECTED
            
            ### lower and selected 
            #demographic folder is neutral NOT selected 
    
            class1_config_lower_filename = '{}/{}/{}/run.{}.est_dfe-site_class1.config.txt'.format(runs_folder, folder, lower, lower)
            f = open(class1_config_lower_filename, 'w')
            f.write("data_path_1\t{} \nsfs_input_file \t    {}/{}/{}/SFS.{}_{}.txt \nest_dfe_results_dir \t   {}/{}/{}/{}_{}_selected \n \nest_dfe_demography_results_file    {}/{}/{}/{}_{}_neutral/est_dfe.out \n site_class    1 \nfold    1 \nepochs    2 \nmean_s_variable    1 \nmean_s    -0.1 \nbeta_variable    1 \nbeta    0.5".format(data_path_1, runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, lower, network_measure))
            f.close()

            #outputs sel_exp_obs_allele_freqs.csv in selected 
            !bash dfe_first_step.sh $class1_config_lower_filename
            

            class1_config_upper_filename = '{}/{}/{}/run.{}.est_dfe-site_class1.config.txt'.format(runs_folder, folder, upper, upper)
            f = open(class1_config_upper_filename, 'w')
            f.write("data_path_1\t{} \nsfs_input_file \t    {}/{}/{}/SFS.{}_{}.txt \nest_dfe_results_dir \t   {}/{}/{}/{}_{}_selected \n \nest_dfe_demography_results_file    {}/{}/{}/{}_{}_neutral/est_dfe.out \n site_class    1 \nfold    1 \nepochs    2 \nmean_s_variable    1 \nmean_s    -0.1 \nbeta_variable    1 \nbeta    0.5".format(data_path_1, runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, upper, network_measure))
            f.close()

            #outputs sel_exp_obs_allele_freqs.csv in selected 
            !bash dfe_first_step.sh $class1_config_upper_filename
            print("done with first step dfe")
            
            #################################################step 1 of dfe is done     
            i+= 1
            print("++++++++++++++++++++++++++++++++++++++*******" + str(i) + "*********************************************************************************" )

            print("second step is prop_muts_in_s_ranges -c $1 -o $2")

            ##upper DFE results
            upper_estdfe_filename = '{}/{}/{}/{}_{}_selected/est_dfe.out'.format(runs_folder, folder, upper, upper, network_measure)
            print(upper_estdfe_filename)
            !bash dfe_second_step.sh $upper_estdfe_filename $runs_folder/$folder/upperDFE.results.txt
            
            lower_estdfe_filename = '{}/{}/{}/{}_{}_selected/est_dfe.out'.format(runs_folder, folder, lower, lower, network_measure)
            
            !bash dfe_second_step.sh $lower_estdfe_filename ./runs/$folder/lowerDFE.results.txt
            print("done second step")
 
            print("------------------------------------------------**************-------------------------")
    
    
            ##for lower 
            alpha_omega_config_lower_file = '{}/{}/{}/run.{}.est_alpha_omega.config.txt'.format(runs_folder, folder, lower, lower)
            f = open(alpha_omega_config_lower_file, 'w')
            f.write("data_path_1\t{} \ndivergence_file \t {}/{}/{}/divergence.{}_{}.txt \nest_alpha_omega_results_file \t {}/{}/{}/est_alpha_omega.out \nest_dfe_results_file \t {}/{}/{}/{}_{}_selected/est_dfe.out \nneut_egf_file \t {}/{}/{}/{}_{}_neutral/neut_egf.out \nsel_egf_file \t {}/{}/{}/{}_{}_selected/sel_egf.out \ndo_jukes_cantor\t 1 \nremove_poly\t 1".format(data_path_1,runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, lower, network_measure, runs_folder, folder, lower, lower, network_measure))
            f.close()
            
            !bash testing.sh $alpha_omega_config_lower_file

            ##for upper 
            alpha_omega_config_upper_file = '{}/{}/{}/run.{}.est_alpha_omega.config.txt'.format(runs_folder, folder, upper, upper)
            f = open(alpha_omega_config_upper_file, 'w')
            f.write("data_path_1\t{} \ndivergence_file \t {}/{}/{}/divergence.{}_{}.txt \nest_alpha_omega_results_file \t {}/{}/{}/est_alpha_omega.out \nest_dfe_results_file \t {}/{}/{}/{}_{}_selected/est_dfe.out \nneut_egf_file \t {}/{}/{}/{}_{}_neutral/neut_egf.out \nsel_egf_file \t {}/{}/{}/{}_{}_selected/sel_egf.out \ndo_jukes_cantor\t 1 \nremove_poly\t 1".format(data_path_1,runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, upper, network_measure, runs_folder, folder, upper, upper, network_measure))
            f.close()
            
            !bash testing.sh $alpha_omega_config_upper_file
            
            get_dfe_outputs('{}/{}/{}/est_alpha_omega.out'.format(runs_folder, folder, upper), upper, network_measure)
            get_dfe_outputs('{}/{}/{}/est_alpha_omega.out'.format(runs_folder, folder, lower), lower, network_measure)
#             get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/upper10th/est_alpha_omega.out', "upper")
#             get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/lower10th/est_alpha_omega.out', "lower")



C:\Users\selsh\repos\Network-Evolution\runs
this is the path C:\Users\selsh\repos\Network-Evolution\runs\run1


'printf' is not recognized as an internal or external command,
operable program or batch file.
'printf' is not recognized as an internal or external command,
operable program or batch file.


C:\Users\selsh\repos\Network-Evolution


dfe_first_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_dfe: No such file or directory
dfe_first_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_dfe: No such file or directory
dfe_first_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_dfe: No such file or directory


done with first step dfe
++++++++++++++++++++++++++++++++++++++*******1*********************************************************************************
second step is prop_muts_in_s_ranges -c $1 -o $2
C:\Users\selsh\repos\Network-Evolution\runs/run1/upper10th/upper10th_irc1080_betweenness_centrality_selected/est_dfe.out


dfe_first_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_dfe: No such file or directory
dfe_second_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/prop_muts_in_s_ranges: No such file or directory


done second step
------------------------------------------------**************-------------------------


dfe_second_step.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/prop_muts_in_s_ranges: No such file or directory
testing.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_alpha_omega: No such file or directory
testing.sh: line 4: /scratch/research/tmp_apps/dfe-alpha-release-2.16/est_alpha_omega: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\selsh\\repos\\Network-Evolution\\runs/run1/upper10th/est_alpha_omega.out'

# getting prop_muts_in_s_ranges all in one file for upper and for lower 

In [1]:
import re
network_measure = "irc1080_betweenness_centrality"
upper = "upper10th"
lower = "lower10th"

# def get_dfe_outputs(filename, t):
#     with open(filename) as f:
#         content = f.readlines()[0]
#         return content

rootdir = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs" 
print(rootdir)
i = 0
file_out_lower = open("dfe_outfile_" + str(lower) +"_" + str(network_measure) + '.csv', 'w')
file_out_upper = open("dfe_outfile_" + str(upper) +"_" + str(network_measure) + '.csv', 'w')
import os
for subdir, dirs, files in os.walk(rootdir):
    for folder in dirs:
        if folder.startswith("r"):
            new_path = os.path.join(subdir, folder)
            print(new_path)
            with open(new_path + "/lowerDFE.results.txt") as f1:
                file_out_lower.write(f1.readlines()[0])
            with open(new_path + "/upperDFE.results.txt") as f2:
                file_out_upper.write(f2.readlines()[0])
file_out_lower.close()
file_out_upper.close()

/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run84
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run56
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run7
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run82
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run24
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run50
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run43
/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run10
/scratch/research/projects/chlamydomonas/network_evolution/ana

In [5]:
# def get_dfe_outputs(filename, t, network_measure):
#     with open(filename) as f:
#         content = f.readlines()[0]
#         print(content)
#         match = re.search('lambda (\d+.\d+) selected_divergence (\d+.\d+) alpha (\d+.\d+) omega_A (\d+.\d+)', content)
#         print("this is match " + str(match))
#         with open(t+"_" +network_measure+'_lambda.txt', 'a') as the_file:
#             the_file.write("{}\n".format(match.group(1)))
#             print("updated file: " + str(the_file))
#         with open(t+"_" +network_measure+'_selected_divergence.txt', 'a') as the_file:
#             the_file.write("{}\n".format(match.group(2)))
#             print("updated file: " + str(the_file))
#         with open(t+"_" +network_measure+'_alpha.txt', 'a') as the_file:
#             the_file.write("{}\n".format(match.group(3)))
#             print("updated file: " + str(the_file))
#         with open(t+"_" +network_measure+'_omega_A.txt', 'a') as the_file:
#             the_file.write("{}\n".format(match.group(4)))
#             print("updated file: " + str(the_file))
#         with open(t+"_" +network_measure+'_all.csv', 'a') as the_file:
#             the_file.write("{},{},{},{}\n".format(match.group(1), match.group(2), match.group(3), match.group(4)))
#             print("updated file: " + str(the_file))
#             ##to add column names
#             df = pd.read_csv(t+"_" +network_measure+'_all.csv', header=None)
#             df.columns = ['lambda_'+ t, 'selected_divergence_'+ t, 'alpha_' +t, 'omega_A_'+t]
#             df.to_csv(t+"_" +network_measure+'_all.csv')
            
# import pandas as pd
# t = "lower10th"
# df = pd.read_csv("lower10th_irc1080_betweenness_centrality_all.csv", header=None)
# df.columns = ['lambda_'+ t, 'selected_divergence_'+ t, 'alpha_' +t, 'omega_A_'+t]
# df.to_csv("lower10th_irc1080_betweenness_centrality_all.csv")

# t = "upper10th"
# df = pd.read_csv("upper10th_irc1080_betweenness_centrality_all.csv", header=None)
# df.columns = ['lambda_'+ t, 'selected_divergence_'+ t, 'alpha_' +t, 'omega_A_'+t]
# df.to_csv("upper10th_irc1080_betweenness_centrality_all.csv")

In [41]:
# def get_dfe_outputs(filename, t, network_measure):
#     with open(filename) as f:
#         content = f.readlines()[0]
#         print(content)
#         match = re.search('lambda (\d+.\d+) selected_divergence (\d+.\d+) alpha (\d+.\d+) omega_A (\d+.\d+)', content)
#         print("this is match " + str(match))
#         with open(t+"_" +network_measure+'_all.csv', 'a') as the_file:
#             the_file.write("{},{},{},{}\n".format(match.group(1), match.group(2), match.group(3), match.group(4)))
#             print("updated file: " + str(the_file))
#             the_file.close
#             ##to add column names
#             df = pd.read_csv(t+"_" +network_measure+'_all.csv', header=None, delimiter=",")
#             print("here is the df ")
#             print(df.head)
#             df.rename(columns={0: 'lambda_'+ t, 1: 'selected_divergence_'+ t, 2:'alpha_' +t, 3:'omega_A_'+t}, inplace=True)
#             df.to_csv('test_with_col.csv', index=False)
            
# get_dfe_outputs("untitled.txt", "lower", "bc")

lambda 4.6 selected_divergence 23.4 alpha 2.4 omega_A 4.5

this is match <_sre.SRE_Match object; span=(0, 57), match='lambda 4.6 selected_divergence 23.4 alpha 2.4 ome>
updated file: <_io.TextIOWrapper name='lower_bc_all.csv' mode='a' encoding='UTF-8'>


ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 5
